In [1]:
# webstie https://nucleisegmentationbenchmark.weebly.com
# paper   https://drive.google.com/file/d/0ByERBiBsEbuTOEJISEpwSkR0SlE/view
import os
import re
import PIL
import time
import random
import shutil
import itertools
import collections
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from xml.dom import minidom
from PIL import Image


In [2]:
TRAIN_PATH = '../orig_data/Tissue images_png/'
XML_PATH   = '../orig_data/Annotations/'
OUT_PATH  = '../orig_data/external_TCGA_train/'  # manual create
#MASK_PATH  = OUT_PATH + '/masks'
#IMG_PATH = OUT_PATH + '/images'

train_ids = next(os.walk(TRAIN_PATH))[2]
xml_ids = next(os.walk(XML_PATH))[2]
print('train_ids = ' + str(len(train_ids)) + '\nxml_ids = ' + str(len(xml_ids)))
IMG_HEIGHT = 1000
IMG_WIDTH = 1000
IMG_CHANNELS = 1

if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH) 


train_ids = 30
xml_ids = 30


In [3]:
def fill_circle(mask_img,vertex_list):
    # max x value for each y
    max_right = {}     
    for i in range(len(vertex_list)):
        current_y = vertex_list[i][0]
        current_x = vertex_list[i][1]
        if current_y not in max_right:
            max_right[current_y] = current_x 
        else:
            max_right[current_y] = max(max_right[current_y],current_x)
    
    for i in range(len(vertex_list)):
        current_y = vertex_list[i][0]
        current_x = vertex_list[i][1]
        mask_img[current_y,current_x:max_right[current_y]] = 255
    
    return mask_img

In [4]:
def check_valid_xy(y, x):
    if x >= IMG_WIDTH: x = IMG_WIDTH-1
    if y >= IMG_HEIGHT: y = IMG_HEIGHT-1                
    if x <= 0: x = 0
    if y <= 0: y = 0
    return y,x

In [5]:
def generate_miss_node(y, x, py, px):
    dist_x = int(x) - px
    dist_y = int(y) - py
    miss_node = []
    if max(abs(dist_y),abs(dist_x)) >= 2 and py != IMG_HEIGHT and px != IMG_WIDTH:
        a = int(x); b = px
        newx = list(map(int,np.linspace(a,b, abs(b-a)+1).tolist()))
        #print('newx => ' + str(newx))
        if len(newx) >= 2: newx.pop(-1)
        if len(newx) >= 2: newx.pop(0)
        a = int(y); b = py
        newy = list(map(int,np.linspace(a,b, abs(b-a)+1).tolist()))
        #print('newy => ' + str(newy))
        if len(newy) >= 2: newy.pop(-1)
        if len(newy) >= 2: newy.pop(0)
        miss_node = list(itertools.product(newy, newx))
    #miss_node = []  # enable this line if you wanna disable this function
    return miss_node

In [28]:
def regions2mask(Resions, xml_idx, mask_folder, area_size):
    mask_contour = np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
    mask_solid   = np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
    
    #pixel_mask_dict = {}
    for region_idx in range(len(Regions)):
        #print('region_idx ' + str(region_idx))
        Region=Regions.item(region_idx)
        verticies=Region.getElementsByTagName('Vertex');
        Region_ID = Region.getAttribute('Id')
        #print('Region_ID ==> ' + str(Region_ID))
        single_mask = np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)    
        vertix_list = []
        px   = IMG_WIDTH
        py   = IMG_HEIGHT        

        for vertexi in range(len(verticies)):
            x=int(float(verticies.item(vertexi).getAttribute('X')))
            y=int(float(verticies.item(vertexi).getAttribute('Y')))
            y, x = check_valid_xy(y, x)

            miss_node = generate_miss_node(y, x, py, px)
            if miss_node != []:
                #print(miss_node)
                vertix_list.extend(miss_node)
                for node_y, node_x in miss_node:
                    mask_contour[node_y,node_x] = 255 
                    single_mask[node_y,node_x] = 255 # signal mask
            px = x
            py = y
            # fill mask contour
            mask_contour[y, x] = 255 # all for one mask   
            single_mask[y, x] = 255 # signal mask
            vertix_list.append([y, x]) 
        # consider relation between first node and last node
        miss_node = generate_miss_node(vertix_list[0][0], vertix_list[0][1], vertix_list[-1][0],vertix_list[-1][1])
        if miss_node != []:
            #print(miss_node)
            vertix_list.extend(miss_node)
            for node_y, node_x in miss_node:
                mask_contour[node_y,node_x] = 255 
                single_mask[node_y,node_x] = 255 # signal mask
        
        # generate folder and file name
        fname = re.sub('.*/|\.xml','',xml_filenames[xml_idx])
        sigle_mask_name = mask_folder + fname + '_' + '{:0>4}'.format(str(Region_ID)) + '.png'
        #print(sigle_mask_name)

        # fill mask circle w/ solid and generate png 
        mask_solid_ = fill_circle(single_mask,vertix_list )
        newImg1= Image.fromarray(mask_solid_,'L')
        newImg1.save(sigle_mask_name,"PNG")
        # merge all single solid-circle to one
        #mask_solid = np.maximum(mask_solid, mask_solid_)
        area_size[region_idx] = (mask_solid_/255).sum()
        if mask_stack.size == 0:
            mask_stack = mask_solid
        else:
            mask_stack = np.vstack((mask_stack, mask_solid))
        return mask_stack

In [7]:
def generate_folder(fname):
    FILE_folder = OUT_PATH + fname + '/'
    MASK_folder = FILE_folder + 'masks/'
    IMG_folder = FILE_folder + 'images/'
    if not os.path.isdir(FILE_folder):
        os.mkdir(FILE_folder)
    if not os.path.isdir(MASK_folder):
        os.mkdir(MASK_folder)
    if not os.path.isdir(IMG_folder):
        os.mkdir(IMG_folder)

    return MASK_folder, IMG_folder
    

In [ ]:
def pixel_based_mask()

In [8]:
#img_filenames = [TRAIN_PATH + f for f in train_ids]
xml_filenames = [XML_PATH + f for f in xml_ids]
#print(img_filenames[0])
print(xml_filenames[0])

../orig_data/Annotations/TCGA-E2-A14V-01Z-00-DX1.xml


In [32]:
start_time = time.time()
region_count = 0

for xml_idx in range(len(xml_filenames)):
    fname = re.sub('.*/|\.xml','',xml_filenames[xml_idx])
    # generate ncecssary folder 
    MASK_folder, IMG_folder = generate_folder(fname)
    # move original png to indivitual folder
    shutil.copy(TRAIN_PATH+fname+'.png',IMG_folder)
    # parsing xml and generate label picture for each region
    root = minidom.parse(xml_filenames[xml_idx])
    Regions=root.getElementsByTagName('Region');
    print(xml_filenames[xml_idx] + '\t' + str(len(Regions)))
    area_size = [0] * len(Regions)
    regions2mask(Regions, xml_idx, MASK_folder, area_size)
    region_count += len(Regions)

    break
print("\ntotal region count is " + str(region_count))
print("--- %s seconds ---" % (time.time() - start_time))    

../orig_data/Annotations/TCGA-E2-A14V-01Z-00-DX1.xml	378

total region count is 378
--- 1.6387755870819092 seconds ---


In [33]:
mask_stack.size

1000000

In [34]:
mask_stack.shape

(1000, 1000)

In [35]:
mask_stack.dtype

dtype('uint8')

In [57]:
a = np.array([[[1,1,1],[1,1,1],[1,1,1]])
print(a)
print(a.shape)

[list([[1, 1, 1], [1, 1, 1]]) list([1, 1, 1])]
(2,)


In [48]:
b = np.array([[[2,2],[2,2]]])
print(b)
print(b.shape)

[[[2 2]
  [2 2]]]
(1, 2, 2)


In [51]:
c = np.vstack((a,b))
print(c)
print(c.shape)

[[[1 1]
  [1 1]]

 [[2 2]
  [2 2]]]
(2, 2, 2)


In [54]:
c[0].shape

(2, 2)

In [55]:
mask_solid

NameError: name 'mask_solid' is not defined